# RecommendKit Quickstart Guide

This notebook demonstrates how to use RecommendKit to build a recommendation system from scratch using simple command-line tools.

## What You'll Learn
1. Generating synthetic data for testing
2. Understanding configuration files
3. Training a model using command-line tools
4. Running inference to get recommendations
5. Exploring the trained model


## 1. Generate Synthetic Dataset

Since we don't upload datasets to GitHub, let's generate a synthetic dataset for testing. This script creates a realistic dataset with perfect correlations between user attributes and item preferences.


In [1]:
# Generate synthetic dataset
!cd datasets/synthetic && python3 generate_correlated_dataset.py --num_users 1000 --num_items 100 --output correlated_dataset.json

print("✅ Synthetic dataset generated!")


🎯 Generating correlated dataset with 1000 users and ~100 items...
✅ Generated 100 items
✅ Generated 1000 users
✅ Generated 13600 interactions

💾 Dataset saved to: correlated_dataset.json

📊 DATASET STATISTICS

🔢 Counts:
   Users: 1000
   Items: 100
   Interactions: 13600

👔 Occupation Distribution:
   artist: 100 users
   chef: 100 users
   doctor: 100 users
   driver: 100 users
   lawyer: 100 users
   mechanical_engineer: 100 users
   nurse: 100 users
   scientist: 100 users
   software_engineer: 100 users
   teacher: 100 users

🌍 Location Distribution:
   Australia: 200 users
   Canada: 200 users
   India: 200 users
   UK: 200 users
   USA: 200 users

📦 Item Category Distribution:
   art: 8 items
   automotive: 8 items
   business: 8 items
   educational: 8 items
   healthcare: 8 items
   kitchen: 10 items
   lab: 6 items
   medical_equipment: 10 items
   tech: 12 items
   tools: 10 items
   weather_cold: 4 items
   weather_hot: 4 items
   weather_rain: 4 items

💬 Interaction Statist

## 2. Explore the Configuration File

RecommendKit uses JSON configuration files to define model architecture. This makes training incredibly simple - just point to a config file! Let's examine the SimpleFusion configuration:


In [14]:
import json

# Load and display the configuration file
config_path = "configs/sample_config_simple_fusion.json"

with open(config_path, 'r') as f:
    config = json.load(f)

print("⚙️  SimpleFusion Configuration File")
print("=" * 60)
print(json.dumps(config, indent=2))


⚙️  SimpleFusion Configuration File
{
  "embedding_dim": 64,
  "loss_type": "bce",
  "user_num_attention_layers": 0,
  "user_num_heads": 1,
  "user_dropout": 0.0,
  "user_use_cls_token": false,
  "user_use_layer_norm": false,
  "user_use_simple_fusion": true,
  "item_num_attention_layers": 0,
  "item_num_heads": 1,
  "item_dropout": 0.0,
  "item_use_simple_fusion": true,
  "interaction_num_attention_layers": 0,
  "interaction_num_heads": 1,
  "interaction_dropout": 0.0,
  "interaction_use_simple_fusion": true,
  "classifier_hidden_dims": [
    64
  ],
  "classifier_dropout": 0.0,
  "text_encoder_config": {
    "model_name": "glove-wiki-gigaword-50",
    "aggregation_strategy": "mean",
    "embedding_dim": 64,
    "num_text_fields": 2
  },
  "categorical_encoder_config": {
    "aggregation_strategy": "separate_concat",
    "hash_vocab_size": 4096,
    "embedding_dim": 64,
    "num_categorical_fields": 1,
    "mlp_hidden_dims": []
  },
  "continuous_encoder_config": {
    "embedding_dim"

## 3. Train the Model

Now let's train the model using the command-line tool. It's as simple as pointing to your data and config file!


In [15]:
# Train the model using command-line tool
!python3 train.py \
  --data_path datasets/synthetic/correlated_dataset.json \
  --config_path configs/sample_config_simple_fusion.json \
  --output_dir models \
  --model_name quickstart_model


🚀 Generic Recommendation System Training
🔄 Loading training configuration...
✅ Configuration loaded from configs/sample_config_simple_fusion.json
🔄 Loading data...
Loaded 13600 interactions
✅ Loaded 1000 users and 100 items
🔄 Loading interactions...
Loaded 13600 real interactions from input data
Positive interactions: 13600
Negative interactions: 0
Positive ratio: 1.000
✅ Loaded 13600 interactions
🔄 Creating model...
Loading Word2Vec model: glove-wiki-gigaword-50
Successfully loaded Word2Vec model with 50 dimensions
Temporal encoder: Creating with item lookup enabled
✅ Model created with 975,361 parameters
✅ Using device: cpu
🔄 Creating data loaders...
Split interactions: 10880 train, 2040 validation, 680 test
Created dataset with 10880 positive and 10880 negative samples
Created dataset with 2040 positive and 2040 negative samples
✅ Data loaders created - Train: 680 batches, Val: 128 batches, Test: 680 interactions
✅ Test interactions saved to models/quickstart_model_test_interactions

## 5. Evaluate Model Performance

Let's evaluate the trained model on the test set to see how well it performs:

In [19]:
# Evaluate the model on test set using classification metrics
import os
!python3 evaluation/evaluate_classification.py \
  --model_dir models \
  --model_name quickstart_model \
  --data_path datasets/synthetic/correlated_dataset.json \
  --test_interactions models/quickstart_model_test_interactions.json \
  --batch_size 32 \
  --threshold 0.5 \
  --output_file classification_results.json

# Display the evaluation results
if os.path.exists("classification_results.json"):
    with open("classification_results.json", 'r') as f:
        results = json.load(f)
    
    print("\n📊 Classification Evaluation Results:")
    print(f"   Total samples: {results['metrics']['num_samples']}")
    print(f"   Accuracy: {results['metrics']['accuracy']:.4f}")
    print(f"   Precision: {results['metrics']['precision']:.4f}")
    print(f"   Recall: {results['metrics']['recall']:.4f}")
    print(f"   F1 Score: {results['metrics']['f1']:.4f}")
    if 'auc_roc' in results['metrics']:
        print(f"   AUC-ROC: {results['metrics']['auc_roc']:.4f}")


📊 Recommendation System Classification Evaluation
🔄 Loading data...
Loaded 13600 interactions
✅ Loaded 1000 users and 100 items
🔄 Loading test interactions...
✅ Loaded 680 test interactions
🔄 Loading model...
✅ Config verified - matches checkpoint structure
🔍 Loading model with categorical_encoder config:
   mlp_hidden_dims: []
   embedding_dim: 64
   num_categorical_fields: 1
Loading Word2Vec model: glove-wiki-gigaword-50
Successfully loaded Word2Vec model with 50 dimensions
Temporal encoder: Creating with item lookup enabled
🔍 Checkpoint categorical encoder MLP layers: [0]
🔍 Model categorical encoder MLP layers: [0]
✅ Model structure verified - matches checkpoint
✅ Pre-initialized user_continuous_encoder MLP (input_dim=2)
✅ Pre-initialized item_continuous_encoder MLP (input_dim=1)
✅ Pre-initialized user_generator.user_fusion.projection (num_features=3)
✅ Pre-initialized item_generator.item_fusion.projection (num_features=3)
✅ Model loaded on device: cpu
🔍 Encoders used: ['image', 'te

## 6. Run Inference

Now let's use the trained model to generate recommendations. The inference script makes it easy to get recommendations for any user!


In [21]:
# Generate recommendations for a user
# The inference script will use the first user by default, or you can specify --user_id
!python3 inference.py \
  --data_path datasets/synthetic/correlated_dataset.json \
  --model_dir models \
  --model_name quickstart_model \
  --user_id 1 \
  --k 10

🚀 Generic Recommendation System - Inference
🔄 Loading data...
Loaded 13600 interactions
✅ Loaded 1000 users and 100 items
🔄 Loading trained model...
✅ Config verified - matches checkpoint structure
🔍 Loading model with categorical_encoder config:
   mlp_hidden_dims: []
   embedding_dim: 64
   num_categorical_fields: 1
Loading Word2Vec model: glove-wiki-gigaword-50
Successfully loaded Word2Vec model with 50 dimensions
Temporal encoder: Creating with item lookup enabled
🔍 Checkpoint categorical encoder MLP layers: [0]
🔍 Model categorical encoder MLP layers: [0]
✅ Model structure verified - matches checkpoint
✅ Pre-initialized user_continuous_encoder MLP (input_dim=2)
✅ Pre-initialized item_continuous_encoder MLP (input_dim=1)
✅ Pre-initialized user_generator.user_fusion.projection (num_features=3)
✅ Pre-initialized item_generator.item_fusion.projection (num_features=3)
🔍 Encoders used during training: ['image', 'text', 'categorical', 'continuous', 'temporal']
🔍 User fusion expects 3 feat

## 7. Next Steps

Congratulations! You've successfully:
1. ✅ Generated synthetic data for testing
2. ✅ Understood the configuration file structure
3. ✅ Trained a model using simple command-line tools
4. ✅ Generated recommendations using inference

### What's Next?

- **Try Different Configurations**: Modify `configs/sample_config_simple_fusion.json` to experiment with different settings
- **Use Your Own Data**: Format your data in the JSON structure shown above
- **Evaluate Your Model**: Use `evaluation/evaluate_classification.py` or `evaluation/evaluate_recommendation.py`
- **Try Attention-Based Fusion**: Use `configs/sample_config_attention_fusion.json` for transformer-based fusion

### Key Takeaways

- **Training is Simple**: Just `python3 train.py --data_path X --config_path Y`
- **Inference is Easy**: Just `python3 inference.py --data_path X --model_dir Y --model_name Z`
- **Config-Driven**: All model settings are in JSON files - no code changes needed!
- **Production Ready**: Saved models can be loaded and used anywhere

### Learn More

Check out the README.md for:
- Detailed architecture explanations
- Advanced customization options
- Evaluation methods
- Integration examples

Happy recommending! 🚀
